In [12]:
from typing import List, Tuple
from random import choices, randint, randrange, random
from collections import namedtuple
Genome = List[int]
Population = List[Genome]
Thing = namedtuple('Thing', ['name', 'value', 'weight'])

things = [Thing('Laptop', 500, 2200),Thing('headphones',150, 160),Thing('Coffee Mug', 60, 350), Thing('Notepad', 40, 333), Thing('Water Bottle', 30, 192)]

def generate_genome(length: int) -> Genome:
    return choices([0,1], k=length) #generate random list of 1's and 0's of the appropriate length (k)
def generate_population(size:int, genome_length: int) -> Population:
    return [generate_genome(genome_length) for _ in range(size)]
def fitness(genome: Genome, things: [Thing], weight_limit: int ) -> int:
    if len(genome) != len(things):
        raise ValueError('Genome and things must be of the same length')
    weight = 0
    value = 0 
    for i, thing in enumerate(things):
        if genome[i] == 1:
            weight += thing.weight
            value += thing.value

            if weight > weight_limit: #if weight is greater than the limit, it should return 0
                return 0 
    return value

def selection_pair(population: Population, fitness_func) -> Population:
    return choices(
        population=population,
        weights=[fitness_func(genome) for genome in population],
        k=2 #tells the function to draw twice, getting a pair
    )

def single_point_crossover(a: Genome, b: Genome) -> Tuple[Genome, Genome]:
    if len(a) != len(b):
        raise ValueError('a and b need to be equal length')
    length = len(a)
    if length < 2:
        return a,b
    p = randint(1, length - 1 )
    return a[0:p] + b[p:], b[0:p] + a[p:]

#flips bits if random num less than probability 
def mutation(genome: Genome, num: int = 1, probability: float = 0.5) -> Genome:
    for _ in range(num):
        index = randrange(len(genome))
        genome[index] = genome[index] if randome() > probability else abs(genome[index] - 1)
    return genome

In [ ]:
def run_evolution(populate_func,fitness_func, fitness_limit: int, selection_func, crossover_func, mutation_func, generation_limit: int = 100) -> Tuple[Population,int]:
    